<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Bias F1 </h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Bias
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])

def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(item,user,damping):
    all_recs = []
    test_data = []
    bias = Bias(items=item, users=user, damping=damping) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('Bias', bias, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla1 = topn.RecListAnalysis()
    rla2 = topn.RecListAnalysis()
    rla1.add_metric(topn.precision)
    rla2.add_metric(topn.recall)
    prec = rla1.compute(all_recs, test_data)
    rec = rla2.compute(all_recs, test_data)
    F1 = ((prec['precision'].values*rec['recall'].values*2)/(prec['precision'].values+rec['recall'].values))
    F1 = np.nan_to_num(F1)
    return [F1.mean(),prec['precision'].mean(),rec['recall'].mean()]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    user = params['user']
    item = params['item']
    damping = params['damping']
    metric = model_trainer(item,user,damping)
    print([item,user,damping,metric[0],metric[1],metric[2]],',')
    return {'loss': -metric[0], 'status': STATUS_OK }

space={'item': hp.choice('item', ['True', 'False']),
       'user': hp.choice('user', ['True', 'False']),
       'damping': hp.uniform('damping', 0, 1)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

['True', 'False', 0.28986599588024275, 0.038960192901424766, 0.029241721854304913, 0.11201555875643622]                
,                                                                                                                      
['True', 'False', 0.11581244600681329, 0.03868198265285198, 0.029013245033112974, 0.11104638687672043]                 
,                                                                                                                      
['True', 'True', 0.6515013396774997, 0.03946973880815993, 0.02961092715231831, 0.11330487253002768]                    
,                                                                                                                      
['False', 'False', 0.24219033845936466, 0.0392816453593031, 0.02943046357615931, 0.11252626573428992]                  
,                                                                                                                      
['False', 'True', 0.039335276957562115, 

['True', 'True', 0.8004046174987927, 0.0421813831024772, 0.031495033112583164, 0.12103502334958617]                    
,                                                                                                                      
['True', 'True', 0.7453123965137423, 0.04161338746061649, 0.031086092715232144, 0.12037746561370169]                   
,                                                                                                                      
['True', 'True', 0.8916488993062852, 0.041900731624899094, 0.03140894039735132, 0.11933457619465301]                   
,                                                                                                                      
['True', 'True', 0.8437747994555864, 0.04275835477001221, 0.03186754966887458, 0.12374145326876583]                    
,                                                                                                                      
['True', 'True', 0.9456422232875622, 0.0

In [4]:
array = [['True', 'False', 0.28986599588024275, 0.038960192901424766, 0.029241721854304913, 0.11201555875643622]                
,                                                                                                                      
['True', 'False', 0.11581244600681329, 0.03868198265285198, 0.029013245033112974, 0.11104638687672043] #107.27
,                                                                                                                      
['True', 'True', 0.6515013396774997, 0.03946973880815993, 0.02961092715231831, 0.11330487253002768]                    
,                                                                                                                      
['False', 'False', 0.24219033845936466, 0.0392816453593031, 0.02943046357615931, 0.11252626573428992]                  
,                                                                                                                      
['False', 'True', 0.039335276957562115, 0.039527231535637294, 0.02956622516556335, 0.11404095631762716]                
,                                                                                                                      
['False', 'True', 0.29535891859865704, 0.03880925212445497, 0.029110927152318302, 0.11093959654383201]                 
,                                                                                                                      
['False', 'True', 0.3727043343632861, 0.03987311071795905, 0.02997847682119241, 0.11303928696941669]                   
,                                                                                                                      
['True', 'False', 0.06808418799544524, 0.03889094485646533, 0.02918708609271558, 0.1116540654540901]                   
,                                                                                                                      
['True', 'False', 0.871558059066281, 0.04256716897818397, 0.03185596026490101, 0.12133651990588039]                    
,                                                                                                                      
['True', 'False', 0.9483576758912972, 0.04338203949908403, 0.0324801324503315, 0.12479904131163146]                    
,                                                                                                                      
['True', 'False', 0.06124255197589745, 0.03872944233957662, 0.029028145695364587, 0.11169880192793209]                 
,                                                                                                                      
['True', 'True', 0.7452267339989014, 0.040787877199009656, 0.030478476821192382, 0.1191185903741869]                   
,                                                                                                                      
['False', 'False', 0.09723857748140552, 0.0389189682227328, 0.029177152317881108, 0.11122653858649162]                 
,                                                                                                                      
['True', 'True', 0.26003753923400075, 0.038717396665799646, 0.029061258278146082, 0.11103704228353402]                 
,                                                                                                                      
['False', 'False', 0.5687890478702882, 0.039832156611574424, 0.029836092715232115, 0.11510831328113935]                
,                                                                                                                      
['True', 'False', 0.3806334061620542, 0.039303818816129286, 0.02950000000000036, 0.11316728045358045]                  
,                                                                                                                      
['False', 'False', 0.5422712259212502, 0.03962758440139431, 0.02973509933774865, 0.113724269963669]                    
,                                                                                                                      
['True', 'False', 0.8145216214913692, 0.04210168875568183, 0.03152152317880827, 0.12009893316518097]                   
,                                                                                                                      
['False', 'True', 0.9568202569466924, 0.042827404885490285, 0.0319751655629142, 0.12331042002891669]                   
,                                                                                                                      
['True', 'True', 0.8919185480116992, 0.04311009204305335, 0.03219536423841078, 0.12347880094838426]                    
,                                                                                                                      
['True', 'True', 0.9824612386983609, 0.043402414136670425, 0.03238245033112614, 0.12513109808106085]                   
,                                                                                                                      
['True', 'True', 0.9771305044160801, 0.043373015962208945, 0.03252649006622549, 0.12296690359799384]                   
,                                                                                                                      
['True', 'True', 0.7154529975345634, 0.039729395473997134, 0.029791390728477182, 0.11429260771737017]                  
,                                                                                                                      
['True', 'True', 0.9301723371182503, 0.04295757896886761, 0.03219039735099374, 0.12189003358949567]                    
,                                                                                                                      
['True', 'True', 0.8062068368507367, 0.0418860412216661, 0.03123509933774871, 0.12129450660289925]                     
,                                                                                                                      
['True', 'False', 0.657405063569767, 0.039508832285545734, 0.029640728476821476, 0.113479023388607]                    
,                                                                                                                      
['True', 'True', 0.9824876207388, 0.04193162595515435, 0.03140066225165594, 0.12011137722689533]                       
,                                                                                                                      
['True', 'False', 0.8326093154326197, 0.04224378844293364, 0.031586092715232106, 0.12042632892584859]                  
,                                                                                                                      
['True', 'False', 0.9959143326983567, 0.04335529259071207, 0.03244701986754994, 0.12476848232738466]                   
,                                                                                                                      
['True', 'True', 0.46890345341608325, 0.040137037474912846, 0.03006291390728505, 0.11579628404191956]                  
,                                                                                                                      
['True', 'False', 0.7450713954970276, 0.041544654580970335, 0.031059602649007017, 0.12089168654718428]                 
,                                                                                                                      
['True', 'False', 0.6266343726712691, 0.03984608720767367, 0.029864238410596378, 0.11416567104450959]                  
,                                                                                                                      
['True', 'True', 0.9047695358685278, 0.04281115008885749, 0.03201986754966927, 0.12280751208620466]                    
,                                                                                                                      
['True', 'False', 0.7008522066045659, 0.040686663492240094, 0.03047350993377524, 0.11815420091508276]                  
,                                                                                                                      
['True', 'True', 0.16691636635525486, 0.03946483996353256, 0.029592715231788486, 0.11361382801349644]                  
,                                                                                                                      
['False', 'True', 0.4253802811412535, 0.03984085846561176, 0.029857615894040002, 0.1137807804121631]                   
,                                                                                                                      
['True', 'False', 0.6001242029671531, 0.040592960402362724, 0.030476821192053336, 0.1146331961682928]                  
,                                                                                                                      
['True', 'True', 0.7866606693563902, 0.04183501985117853, 0.03129470198675525, 0.12035938715434889]                    
,                                                                                                                      
['False', 'False', 0.8414058904747707, 0.04217252505669419, 0.031476821192053396, 0.12073762269124964]                 
,                                                                                                                      
['True', 'True', 0.5278621312327028, 0.0396612860793782, 0.02980463576158973, 0.11250884293542147]                     
,                                                                                                                      
['True', 'False', 0.6774487932045821, 0.04030575706539622, 0.03030960264900699, 0.11463713985583383]                   
,                                                                                                                      
['False', 'True', 0.3238338711365343, 0.03931431622475222, 0.02949503311258309, 0.1135706117956641]                    
,                                                                                                                      
['True', 'False', 0.0005143056068148133, 0.038575535597183234, 0.02896026490066268, 0.11088040319921372]               
,                                                                                                                      
['True', 'False', 0.21146426497636245, 0.038769138646985786, 0.02907615894039772, 0.11083283068900993]                 
,                                                                                                                      
['False', 'True', 0.9291203144618421, 0.042776021247081145, 0.031958609271523546, 0.12219790472751149]                 
,                                                                                                                      
['True', 'False', 0.7697710315618882, 0.0415459414652746, 0.030980132450331536, 0.1204264053602289]                    
,                                                                                                                      
['True', 'True', 0.9981050715480255, 0.04322032910705435, 0.03232947019867569, 0.124362501896294]                      
,                                                                                                                      
['False', 'False', 0.8624178479806117, 0.04250990971424166, 0.03178476821192085, 0.12192751110665109]                  
,                                                                                                                      
['True', 'True', 0.9432007786695819, 0.042372144230086735, 0.03170033112582819, 0.120687432115706]                     
,                                                                                                                      
['True', 'False', 0.5814773510503051, 0.03893605646346539, 0.0292715231788082, 0.11085171892676399]                    
,                                                                                                                      
['False', 'True', 0.48318685143716883, 0.03976790185483563, 0.029778145695364668, 0.11414596474874995]                 
,                                                                                                                      
['True', 'False', 0.7246534292647275, 0.04063068373863369, 0.030415562913907658, 0.11728406581443883]                  
,                                                                                                                      
['True', 'True', 0.8810459661387219, 0.042767707241662775, 0.032001655629139404, 0.12312095970961297]                  
,                                                                                                                      
['False', 'False', 0.37768672340445286, 0.03910597197786032, 0.02926655629139114, 0.11266005312479876]                 
,                                                                                                                      
['True', 'True', 0.1444561222171159, 0.039081943386544506, 0.029367549668874542, 0.11110452885752085]                  
,                                                                                                                      
['True', 'False', 0.42799404155755416, 0.03977758137845884, 0.029869205298013617, 0.1139642078185698]                  
,                                                                                                                      
['True', 'True', 0.6221330627433121, 0.040007971322636185, 0.030008278145695687, 0.1147019918436878]                   
,                                                                                                                      
['True', 'False', 0.9055581958075101, 0.04285980308424575, 0.032009933774834735, 0.12276046776005056]                  
,                                                                                                                      
['False', 'True', 0.3121190395425735, 0.039076395430460556, 0.029264900662252097, 0.11243960824696346]                 
,                                                                                                                      
['True', 'False', 0.7778157784908533, 0.041619593791868456, 0.031034768211920805, 0.12076463644477425]                 
,                                                                                                                      
['True', 'True', 0.8242563183443687, 0.0424044673602193, 0.03162251655629165, 0.12231729350381047]                     
,                                                                                                                      
['True', 'False', 0.9615814879776801, 0.04285071981292183, 0.0321059602649009, 0.12232423549020634]                    
,                                                                                                                      
['True', 'True', 0.9950670765239529, 0.04262898469276486, 0.03198509933774874, 0.12143481274298794]                    
,                                                                                                                      
['False', 'False', 0.5438999400397526, 0.03967593573183463, 0.02983112582781505, 0.11337596331147226]                  
,                                                                                                                      
['True', 'True', 0.8601434799500043, 0.04272165704052514, 0.03187251655629179, 0.1228351881354013]                     
,                                                                                                                      
['True', 'True', 0.9194208121835362, 0.04292322307053341, 0.03203807947019884, 0.12343279063990699]                    
,                                                                                                                      
['True', 'True', 0.9687872029705198, 0.042754463032228775, 0.03198178807947054, 0.12227662506569432]                   
,                                                                                                                      
['True', 'True', 0.6840974665289563, 0.040154976428546846, 0.030018211920530207, 0.11656049170590614]                  
,                                                                                                                      
['True', 'True', 0.8004046174987927, 0.0421813831024772, 0.031495033112583164, 0.12103502334958617]                    
,                                                                                                                      
['True', 'True', 0.7453123965137423, 0.04161338746061649, 0.031086092715232144, 0.12037746561370169]                   
,                                                                                                                      
['True', 'True', 0.8916488993062852, 0.041900731624899094, 0.03140894039735132, 0.11933457619465301]                   
,                                                                                                                      
['True', 'True', 0.8437747994555864, 0.04275835477001221, 0.03186754966887458, 0.12374145326876583]                    
,                                                                                                                      
['True', 'True', 0.9456422232875622, 0.04350326069962998, 0.03257284768211954, 0.1243224299327345]                     
,                                                                                                                      
['True', 'True', 0.9364421696482659, 0.04341404763492712, 0.03250165562913948, 0.12417712755799047]                    
,                                                                                                                      
['True', 'True', 0.652706558758773, 0.040234859172932035, 0.030144039735099878, 0.11682364457418608]                   
,                                                                                                                      
['True', 'True', 0.9981770184115988, 0.04283987952519352, 0.03214238410596053, 0.12186747962157636]                    
,                                                                                                                      
['True', 'True', 0.7492477131980387, 0.04075144871545557, 0.030354304635762004, 0.11826181449367719]                   
,                                                                                                                      
['True', 'True', 0.9314662843221985, 0.04326884894725391, 0.03232947019867573, 0.1237419793324923]                     
,                                                                                                                      
['True', 'True', 0.9515329139778936, 0.04307264523439244, 0.0322168874172189, 0.12273690571191426]                     
,                                                                                                                      
['True', 'True', 0.8669729869791359, 0.04276800001124518, 0.031932119205298336, 0.12412406085502928]                   
,                                                                                                                      
['False', 'True', 0.7171410419451788, 0.03958251251591709, 0.029635761589404327, 0.11416142087991474]                  
,                                                                                                                      
['True', 'True', 0.9789338148520792, 0.042898344444734435, 0.032231788079470525, 0.12138684236296361]                  
,                                                                                                                      
['True', 'True', 0.8171567584876259, 0.04180188612265193, 0.03127317880794748, 0.1199394308584282]                     
,                                                                                                                      
['True', 'True', 0.9149990451747935, 0.04291668810908245, 0.03213079470198713, 0.1225284823735738]                     
,                                                                                                                      
['False', 'True', 0.5162723745152781, 0.03962524433379165, 0.02973344370860966, 0.112802977009804]                     
,                                                                                                                      
['True', 'True', 0.766754858704535, 0.04175161917629671, 0.031149006622516913, 0.12063549147885785]                    
,                                                                                                                      
['True', 'True', 0.030651856275609457, 0.039071743540573994, 0.02926655629139111, 0.11175118925531308]                 
,                                                                                                                      
['True', 'True', 0.6213196927708011, 0.040194062865841006, 0.030094370860927484, 0.11546914773698093]                  
,                                                                                                                      
['True', 'True', 0.2095068845395771, 0.039170062482290866, 0.029362582781457307, 0.11203118373281293]                  
,                                                                                                                      
['False', 'True', 0.43992932111990807, 0.03971799301824102, 0.029817880794702347, 0.11458213577472742]                 
,                                                                                                                      
['True', 'True', 0.7972481364653745, 0.041439996978590986, 0.030879139072848005, 0.11999109300077326]                  
,                                                                                                                      
['True', 'True', 0.6850662874813607, 0.03999956208858439, 0.030064569536424158, 0.11339964506713542]                   
,                                                                                                                      
['True', 'True', 0.34922055500900173, 0.039858134083079394, 0.029885761589404425, 0.1149529600624025]                  
,                                                                                                                      
['False', 'True', 0.5887024373062908, 0.03922294546721665, 0.029399006622517036, 0.11305837315510635]                  
,                                                                                                                      
['True', 'True', 0.8524761801451683, 0.04232527009984794, 0.03162748344370892, 0.12076461984518265]                    
,                                                                                                                      
['True', 'True', 0.8842433505902985, 0.04235765065195419, 0.0316920529801328, 0.12120723003847249]                     
,                                                                                                                      
['True', 'True', 0.9423574828972753, 0.0424851437675439, 0.03173841059602688, 0.12224557603083258]                     
,                                                                                                                      
['True', 'True', 0.2794692516236156, 0.03898709634229327, 0.029263245033112992, 0.11141200769861433]                   
,                                                                                                                      
['False', 'True', 0.8345615523136546, 0.04209122925818248, 0.03139238410596066, 0.12221080946471109]                   
,                                                                                                                      
['True', 'True', 0.8991103800055007, 0.042522695057997766, 0.03186589403973536, 0.1209387199434351] ]

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['item','user','damping','F1','Precision','Recall']
metric.head(5)

,item,user,damping,F1,Precision,Recall
0,True,False,0.289866,0.038960,0.029242,0.112016
1,True,False,0.115812,0.038682,0.029013,0.111046
2,True,True,0.651501,0.039470,0.029611,0.113305
3,False,False,0.242190,0.039282,0.029430,0.112526
4,False,True,0.039335,0.039527,0.029566,0.114041


In [6]:
metric.sort_values(by=['F1'],ascending=False).head(10)

,item,user,damping,F1,Precision,Recall
72,True,True,0.945642,0.043503,0.032573,0.124322
73,True,True,0.936442,0.043414,0.032502,0.124177
20,True,True,0.982461,0.043402,0.032382,0.125131
9,True,False,0.948358,0.043382,0.032480,0.124799
21,True,True,0.977131,0.043373,0.032526,0.122967
28,True,False,0.995914,0.043355,0.032447,0.124768
77,True,True,0.931466,0.043269,0.032329,0.123742
46,True,True,0.998105,0.043220,0.032329,0.124363
19,True,True,0.891919,0.043110,0.032195,0.123479
78,True,True,0.951533,0.043073,0.032217,0.122737


In [7]:
metric.to_csv('metric_F1.csv', index=False)

In [2]:
metric = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Algoritmos\\Bias\\results\\metric_F1.csv')
metric

,item,user,damping,F1,Precision,Recall
0,True,False,0.775844,0.056720,0.038367,0.174842
1,True,False,0.078141,0.053133,0.035748,0.164216
2,False,True,0.565503,0.055032,0.037296,0.168978
3,True,True,0.215608,0.053815,0.036384,0.164570
4,True,False,0.694015,0.056122,0.037985,0.170535
...,...,...,...,...,...,...
95,False,False,0.657520,0.054317,0.036829,0.167053
96,False,False,0.712235,0.054658,0.037147,0.164354
97,False,False,0.302411,0.053453,0.036225,0.162399
98,False,True,0.998017,0.054334,0.036914,0.166061
